##Input CSV

In [0]:
import pandas as pd
import numpy as np

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
dataTestFileID = "1uX1VLYetZIBGlipFVRixpIi8PHDCzkVh"
dataTrainFileID = "1wrBQQzM_VKdTHZM5BdfQz86LsYU1cB1e"
dataTrain2FileID = "12HMKD7sdBV9lIyQMnwOEEtDDv0-dAzK6"

In [0]:
fileID = dataTrainFileID
downloaded = drive.CreateFile({'id': fileID})

In [0]:
rawData = []
tweets=downloaded.GetContentString().split('\n')
print(tweets)
for tweet in tweets:
  data = tweet.split("\t")    
  rawData.append(data)

#Turn rawData into DataFrame  
rawData = pd.DataFrame(rawData)

###Only take the first column (Verdict) and the second column (Text)

In [0]:
cols = ["Text", "Verdict"]
inputData = rawData.iloc[:,[2,1]] 
inputData.columns = cols

In [0]:
inputData.info()

###Delete Rows that have Nil

In [0]:
inputData = inputData.dropna()
inputData.info()

###Replace Verdict with numerical values
positive = 1
negative = 2
neutral = 0

In [0]:
inputData["Verdict"] = inputData["Verdict"].replace(to_replace=["positive","neutral","negative"], value=[1,2,0])
inputData

##Clean the text

In [0]:
!pip install pyspellchecker
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from spellchecker import SpellChecker
from nltk.tokenize import ToktokTokenizer
from nltk.stem.snowball import SnowballStemmer
import re

import nltk
nltk.download('stopwords')

In [0]:
#Reduce repitition of characters, because it is well known that 
#English words ussually has only 2 repeated characters
def reduceLengthening(text):
  pattern = re.compile(r"(.)\1{2,}")
  return pattern.sub(r"\1\1", text)

In [0]:
toktok = ToktokTokenizer().tokenize
stemmer = SnowballStemmer('english')

def cleanText(text):
  mentionPat = r'@\w+'
  linkPat = r'(http|https)?://[\w./]+'
  combinedPat = r'|'.join((mentionPat,linkPat))
  stop_words = set(stopwords.words('english'))
  
  #Clean text from mention and links
  withoutML = re.sub(combinedPat, '', text)
  
  #Clean text from unknown character (unciode = u\ufffd)
  withoutUC = withoutML.replace(u"\ufffd", "?")
  
  #Clean text from non text characters
  textCharOnly = re.sub("[^A-Za-z]"," ",withoutUC)
  
  #lowercase only
  lowercase = textCharOnly.lower()
  
  #reduce lengthening of word (amazzziiingg -> amazziingg)
  reducedLength = reduceLengthening(lowercase)
  
#   #spell checking     
#   spell = SpellChecker() 
#   corrected = " ".join([spell.correction(word) for word in cleanWords])
  
  #Tokenize
  tokenizedWords = toktok(reducedLength)

  #remove stopwords  
  withoutSW = [word for word in tokenizedWords if not (word in stop_words)]
  
  #stemming
  stemmed = [stemmer.stem(word) for word in withoutSW]
  
  clean = " ".join(stemmed)
  
  return clean

###Apply the cleaning method to the text

In [0]:
inputData["Text"] = inputData["Text"].apply(cleanText)
inputData

##Save the clean tweet to drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
if (fileID == dataTestFileID):
  inputData.to_csv('/content/gdrive/My Drive/NLP-SA/clean_data_test.csv')
elif (fileID == dataTrainFileID):
  inputData.to_csv('/content/gdrive/My Drive/NLP-SA/clean_data_train.csv')